In [ ]:
#%matplotlib inline

In [1]:
#!conda install spacy -y

In [21]:
#!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 4.1MB/s ta 0:00:011   11% |███▊                            | 4.3MB 1.7MB/s eta 0:00:20    14% |████▋                           | 5.4MB 14.9MB/s eta 0:00:03    18% |██████                          | 7.1MB 4.7MB/s eta 0:00:07    20% |██████▌                         | 7.6MB 15.0MB/s eta 0:00:02    21% |███████                         | 8.2MB 4.8MB/s eta 0:00:07    22% |███████▎                        | 8.4MB 3.9MB/s eta 0:00:08    27% |████████▋                       | 10.1MB 8.1MB/s eta 0:00:04    29% |█████████▍                      | 11.0MB 8.8MB/s eta 0:00:03    33% |██████████▉                     | 12.6MB 8.7MB/s eta 0:00:03    41% |█████████████▏                  | 15.3MB 7.4MB/s eta 0:00:03    42% |█████████████▋                  | 15.8MB 8.0MB/s eta 0:00:03    44% |██████████████▏                 | 16.6MB 5.1MB/s eta 0:00:05    45% |██████████████▋                 | 17.1MB 2.5MB/s eta 0:00:08    67% |█████████████████████▌       

In [2]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob
import re
import spacy
from spacy import displacy

In [3]:
spacy.__version__

'2.0.2'

In [4]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [5]:
rad_data = \
pd.read_sql("""SELECT noteevents.subject_id, 
                      noteevents.hadm_id,
                      noteevents.text 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT' LIMIT 10000""",conn)
rad_data.head(5)

,subject_id,hadm_id,text
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...


In [6]:
rad_data.shape

(10000, 3)

In [7]:
print(rad_data.iloc[20]["text"])




     DATE: [**3352-7-6**] 10:20 AM
     CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 1631**]
     Reason: fever- eval for infection
     Admitting Diagnosis: NON-HODGKIN'S LYMPHOMIA\BONE MARROW TRANSPLANT
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      54 year old man with T-cell lymphoma, neutropenia
     REASON FOR THIS EXAMINATION:
      fever- eval for infection
     ______________________________________________________________________________
                                     FINAL REPORT
     INDICATION:  Neutropenia and fever.
     
     COMPARISON: [**6-23**], [**6-6**] and [**3352-6-3**].
     
     CHEST AP PORTABLE RADIOGRAPH:  The cardiac, mediastinal and hilar contours are
     unremarkable.  The left lung is clear.  There is a somewhat well circumscribed
     opacity immediately overlying the catheter which runs along the lateral right
 

### Write a function that returns the impression section of a report

#### Hints

* Not every report will have an impression section
* "INTERPRETATION" and "CONCLUSIONS" might be synonyms for "IMPRESSION"

In [8]:
def get_impression(report):
    headings = ["INTERPRETATION:", "CONCLUSIONS:", "IMPRESSION:"]
    for heading in headings:
        if heading in report:
            return report.split(heading)[1]
    return report    


#### In how many reports did we find an impression section?

In [9]:
def count_impression(report):
    headings = ["INTERPRETATION:", "CONCLUSIONS:", "IMPRESSION:"]
    for heading in headings:
        if heading in report:
            return True
    return False


In [10]:
sum([count_impression(report) for report in rad_data["text"]])

8189

### Define Regular expressions for data cleansing

* Write a regular expression to replace dates in the reports with ``[**DATE**]``
* Write a regular expression to replace times in the reports with ``[**TIME**]``
* Write a regular expression to replaces digits with "d", (e.g. "43 cc" would become "dd cm")

#### Hints

* Look at some sample reports to see what dates and times look like in the reports
* What order would you need to apply the regular expressions?

In [11]:
for i in range(10):
    print(rad_data.iloc[i]['text'])
    print("*"*42,"\n\n")




     DATE: [**2644-1-17**] 10:53 AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) 12569**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: 15
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over 90 **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over 90 **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [12]:
date = re.compile(r"""\d""")

In [13]:
print(rad_data.iloc[0]["text"])




     DATE: [**2644-1-17**] 10:53 AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) 12569**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: 15
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over 90 **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over 90 **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [14]:
print(rad_data.iloc[0]["text"])




     DATE: [**2644-1-17**] 10:53 AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) 12569**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: 15
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over 90 **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over 90 **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [15]:
digits = re.compile(r"""\d""")

In [16]:
print(digits.sub("d", rad_data.iloc[0]["text"]))




     DATE: [**dddd-d-dd**] dd:dd AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) ddddd**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: dd
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over dd **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over dd **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [17]:
rad_data["impression"] = \
rad_data.apply(lambda row: digits.sub("d", get_impression(row["text"])).lower(), axis=1)

In [18]:
rad_data.head()

,subject_id,hadm_id,text,impression
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,\n\n\n date: [**dddd-d-dd**] dd:dd am\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,\n\n\n date: [**dddd-d-dd**] dd:dd am\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...,stable appearance of right parietal lobe and...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...,\n \n cardiomegaly and mild chf. naso...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...,\n \n marked improvement in left perih...


In [19]:
print(rad_data.iloc[20]["impression"])


     d)  there is a somewhat well-circumscribed but subtle opacity immediately
     adjacent to the external portion of the catheter overlying the right upper
     lobe.  this may be related to the catheter itself but it is difficult to
     exclude an underlying opacity in this region.  correlation with physical exam
     recommended.




In [36]:
import spacy
from spacy import displacy

doc = nlp(u'Rats are various medium-sized, long-tailed rodents.')
displacy.render(doc, style='dep', jupyter=True)

/opt/conda/lib/python3.5/runpy.py:184: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", mod_spec)
/opt/conda/lib/python3.5/runpy.py:184: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", mod_spec)
/opt/conda/lib/python3.5/runpy.py:184: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", mod_spec)
/opt/conda/lib/python3.5/runpy.py:184: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", mod_spec)


In [73]:
doc2 = nlp(rad_data.iloc[3987]["impression"])
displacy.render(doc2, style='ent', jupyter=True)

In [76]:
for s in doc2.sents:
    #print(" ".join(s))
    #print(s.words)
    print(s)
    print(s.string)
    print("-"*42)

 decrease in the size of the left-sided pleural effusion.
 decrease in the size of the left-sided pleural effusion. 
------------------------------------------
et tube
     tip approximately d cm above the carina.
et tube
     tip approximately d cm above the carina. 
------------------------------------------
persistent retrocardiac opacity
     representing atelectasis vs. consolidation.


persistent retrocardiac opacity
     representing atelectasis vs. consolidation.


------------------------------------------


In [78]:
s.lower_

'persistent retrocardiac opacity\n     representing atelectasis vs. consolidation.'

In [68]:
rclean = re.compile(r"""\W+""")

In [69]:
rclean.sub(" ",s.string)

'correlation with physical exam recommended '

In [48]:
for word in s:
    print("%s->%s"%(word, word.lemma_))


     ->
     
d->have
)->)
 -> 
there->there
is->be
a->a
somewhat->somewhat
well->good
-->-
circumscribed->circumscribe
but->but
subtle->subtle
opacity->opacity
immediately->immediately

     ->
     
adjacent->adjacent
to->to
the->the
external->external
portion->portion
of->of
the->the
catheter->catheter
overlying->overlie
the->the
right->right
upper->upper

     ->
     
lobe->lobe
.->.
 -> 


In [28]:
print(""" d)  there is a somewhat well-circumscribed but subtle opacity immediately
     adjacent to the external portion of the catheter overlying the right upper
     lobe.  this may be related to the catheter itself but it is difficult to
     exclude an underlying opacity in this region.  correlation with physical exam
     recommended.""")

 d)  there is a somewhat well-circumscribed but subtle opacity immediately
     adjacent to the external portion of the catheter overlying the right upper
     lobe.  this may be related to the catheter itself but it is difficult to
     exclude an underlying opacity in this region.  correlation with physical exam
     recommended.


In [30]:
!python -m spacy download custom_ner_model


    Compatibility error
    No compatible model found for 'custom_ner_model' (spaCy v2.0.2).



In [32]:
text = """But Google is starting from behind. The company made a late push
into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa
software, which runs on its Echo and Dot devices, have clear leads in
consumer adoption."""

nlp = spacy.load('en')
doc = nlp(text)
displacy.render(doc, style='ent', jupyter=True)

In [35]:
text = """But Google is starting from behind. The company made a late push
into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa
software, which runs on its Echo and Dot devices, have clear leads in
consumer adoption."""

nlp = spacy.load('en')
doc = nlp(rad_data.iloc[2000]["impression"])
displacy.render(doc, style='ent', jupyter=True)

### How many unique words occur in the corpus?

#### Hint

1. Use TextBlob
1. Put all the reports into a single string

#### I got 8658

In [ ]:
#" ".join(rad_data["impression"])

In [ ]:
unique_impression_words = set(TextBlob(" ".join([report for report in rad_data["impression"]])).words)
len(unique_impression_words)



In [ ]:
#unique_impression_words

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS
STOPWORDS

In [ ]:
my_stop_words = frozenset(["a", "am", "an", "and", "are", "as", "at", "be", "for", "is", "the", "is", "of", "which", ])

## Create a single string with all the reports

#### Hints, etc.
* Use List Comprehension
* Use string joins
* Iterate over the rows of the data frame

### Define a vector space for the radiology corpus

#### Hints

1. How would you build a corpus from words only occuring more than N times?

### Create a new column named `"impression no stops"` where [stop words](https://en.wikipedia.org/wiki/Stop_words) have been dropped from the impression

* The gensim package has stop words defined (``from gensim.parsing.preprocessing import STOPWORDS``

#### Hints
1. Do you agree with dropping all the stop words?
1. How could we create a new stopwords frozen set absent the terms we wan't to keep (double negative?)
1. You could use a regular expressions substitution or token the report first and operate on the list of words.

In [ ]:
rad_data["impression no stops"] = \
rad_data.apply(lambda row: " ".join([word for word in row["impression"].split() \
                            if word not in my_stop_words]), axis=1)

In [ ]:
rad_data.iloc[0]["impression"]

In [ ]:
rad_data.iloc[0]["impression no stops"]

In [ ]:
unique_impression_words = set(TextBlob(" ".join(rad_data["impression no stops"])).words)


In [ ]:
word_map = dict(zip(unique_impression_words,range(len(unique_impression_words))))

In [ ]:
len(word_map)

In [ ]:
word_map